In [1]:
import sqlalchemy
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float
import pandas as pd
from warnings import filterwarnings
import pymysql
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
filterwarnings('ignore', category=pymysql.Warning)
import os
import csv

In [3]:
#connect to the measurement csv file:
clean_measurement_csv = os.path.join("csv_folder", "clean_hawaii_measurements.csv")
clean_measurement_df = pd.read_csv(clean_measurement_csv, dtype=object)
clean_measurement_df.head(2)

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0,63


In [4]:
#connect to the station csv file:
clean_station_csv = os.path.join("csv_folder", "clean_hawaii_stations.csv")
clean_station_df = pd.read_csv(clean_station_csv, dtype=object)
clean_station_df.head(2)


,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6


In [5]:
engine = create_engine("sqlite:///database/hawaii.sqlite")

In [6]:
clean_measurement_df.to_sql(con=engine, name='w_measurement', if_exists='replace', index=False)

In [7]:
clean_station_df.to_sql(con=engine, name='w_station', if_exists='replace', index=False)

In [8]:
#weather measurement and station class
class Measurement(Base):
    __tablename__ = "w_measurement"
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)
    
class Station(Base):
    __tablename__ = "w_station"
    
    station = Column(String, primary_key=True)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    

In [9]:
#nonbondlmwpwg
data = clean_measurement_df.to_dict(orient='records')
data[0]

{'date': '2010-01-01', 'prcp': '0.08', 'station': 'USC00519397', 'tobs': '65'}

In [10]:
#nonbondlmwpwg
data = clean_station_df.to_dict(orient='records')
data[0]

{'elevation': '3',
 'latitude': '21.2716',
 'longitude': '-157.8168',
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [11]:
#nsklnffsono
Base.metadata.create_all(engine)
session = Session(bind=engine)

In [12]:
# Use the Base class to reflect the database tables
Base = automap_base()
Base.prepare(engine, reflect=True)

In [13]:
engine.execute('select * from w_measurement limit 10').fetchall()

[('USC00519397', '2010-01-01', '0.08', '65'),
 ('USC00519397', '2010-01-02', '0', '63'),
 ('USC00519397', '2010-01-03', '0', '74'),
 ('USC00519397', '2010-01-04', '0', '76'),
 ('USC00519397', '2010-01-07', '0.06', '70'),
 ('USC00519397', '2010-01-08', '0', '64'),
 ('USC00519397', '2010-01-09', '0', '68'),
 ('USC00519397', '2010-01-10', '0', '73'),
 ('USC00519397', '2010-01-11', '0.01', '64'),
 ('USC00519397', '2010-01-12', '0', '61')]

In [14]:
engine.execute('select * from w_station limit 10').fetchall()

[('USC00519397', 'WAIKIKI 717.2, HI US', '21.2716', '-157.8168', '3'),
 ('USC00513117', 'KANEOHE 838.1, HI US', '21.4234', '-157.8015', '14.6'),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', '21.5213', '-157.8374', '7'),
 ('USC00517948', 'PEARL CITY, HI US', '21.3934', '-157.9751', '11.9'),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', '21.4992', '-158.0111', '306.6'),
 ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', '21.33556', '-157.71139', '19.5'),
 ('USC00519281', 'WAIHEE 837.5, HI US', '21.45167', '-157.84889', '32.9'),
 ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', '21.3152', '-157.9992', '0.9'),
 ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', '21.3331', '-157.8025', '152.4')]

In [15]:
inspector = inspect(engine)
for table_name in inspector.get_table_names():
   print('table name = ' + table_name)
   for column in inspector.get_columns(table_name):
        try:
           print(chr(9) + column['name'], column['type'])
        except Exception as e:
            print(f"{column} was skipped")
            continue

table name = w_measurement
	station TEXT
	date TEXT
	prcp TEXT
	tobs TEXT
table name = w_station
	station TEXT
	name TEXT
	latitude TEXT
	longitude TEXT
	elevation TEXT
